# TP parte 1

In [74]:
import pandas as pd
import numpy as np
import requests
from collections import Counter
import plotly.express as px
from matplotlib import pyplot as plt
from pandas_profiling import ProfileReport
from matplotlib import pyplot as plt

from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import (
    KBinsDiscretizer,
    LabelEncoder,
    MinMaxScaler,
    Normalizer,
    OneHotEncoder,
    OrdinalEncoder,
    PowerTransformer,
    RobustScaler,
    StandardScaler,
)

ModuleNotFoundError: No module named 'pandas_profiling'

In [75]:
with requests.get(
    "https://docs.google.com/spreadsheets/d/1-DWTP8uwVS-dZY402-dm0F9ICw_6PNqDGLmH0u8Eqa0/export?format=csv"
) as r, open("impuestos.csv", "wb") as f:
    for chunk in r.iter_content():
        f.write(chunk)

# Introduccion 

El presente trabajo analizara determinados datos de personas que pagan impuestos brindados por una agencia gubernamental de recaudación de impuestos la FIUFIP con el objetivo de determinar si una persona posee altos ingresos o bajos ingresos.
La agencia tiene como principal objetivo determinar que individuos evaden los impuestos.

Los objetivos para este trabajo son
   * poder comprender los datos (viendolos y graficandolos)
   * ver como se relacionan entre ellos 
   * determinar patrones a partir de los graficos

Abrimos el data frame

In [76]:
df = pd.read_csv("impuestos.csv")

# Ingenieria de feautures  
Verificamos la calidad de los datos antes de visualizarlos.  

## Verificacion de datos faltantes
Primero verificamos aquellos valores que sean nulos. Con info() verificamos si los hay.

In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 13 columns):
 #   Column                                      Non-Null Count  Dtype 
---  ------                                      --------------  ----- 
 0   anios_estudiados                            32561 non-null  int64 
 1   barrio                                      31978 non-null  object
 2   categoria_de_trabajo                        30725 non-null  object
 3   edad                                        32561 non-null  int64 
 4   educacion_alcanzada                         32561 non-null  object
 5   estado_marital                              32561 non-null  object
 6   ganancia_perdida_declarada_bolsa_argentina  32561 non-null  int64 
 7   genero                                      32561 non-null  object
 8   horas_trabajo_registradas                   32561 non-null  int64 
 9   religion                                    32561 non-null  object
 10  rol_familiar_registrad

Vemos que hay valores que nos faltan en la columna de trabajo, por ejemplo. Los pasamos todos a nan.

In [78]:
#aniadir mas chequeos??
df.fillna(np.nan, inplace = True)
df.isnull().sum()

anios_estudiados                                 0
barrio                                         583
categoria_de_trabajo                          1836
edad                                             0
educacion_alcanzada                              0
estado_marital                                   0
ganancia_perdida_declarada_bolsa_argentina       0
genero                                           0
horas_trabajo_registradas                        0
religion                                         0
rol_familiar_registrado                          0
tiene_alto_valor_adquisitivo                     0
trabajo                                       1843
dtype: int64

Dado que tenemos tres columnas con datos faltantes, tenemos que resolverlos de alguna manera.  
Para esto, tomamos las distintas decisiones:  
  *  Barrio: completamos con la media
  *  Categoria_de_trabajo: reempazamos los valores faltantes por "No respondio". falta justificacion
  *  Trabajo: completamos con la media

In [83]:
df['categoria_de_trabajo'] = df['categoria_de_trabajo'].replace(np.nan, 'No inscripto')
df['trabajo'] = df['trabajo'].replace(np.nan, 'desempleado')#lo agreguge recien
categorias = df['categoria_de_trabajo'].value_counts(dropna=False)
trabajos = df['trabajo'].value_counts(dropna=False)
#categorias.head()
#trabajos.head()

Queremos observar cuantas personas no estan inscriptas ni tienen trabajo.

In [88]:
df.loc[(df['categoria_de_trabajo'] == 'No inscripto') & (df['trabajo'] == 'desempleado')].head()

anios_estudiados   barrio categoria_de_trabajo  edad educacion_alcanzada  \
27                 14    Boedo         No inscripto    54              5_anio   
61                  8      NaN         No inscripto    32           7-8_grado   
69                 14  Palermo         No inscripto    25              5_anio   
77                 10  Palermo         No inscripto    67              1_anio   
106                10  Palermo         No inscripto    17              1_anio   

         estado_marital  ganancia_perdida_declarada_bolsa_argentina  genero  \
27     matrimonio_civil                                           0  hombre   
61   pareja_no_presente                                           0  hombre   
69       sin_matrimonio                                           0  hombre   
77     matrimonio_civil                                           0  hombre   
106      sin_matrimonio                                       34095   mujer   

     horas_trabajo_registradas      religion rol_familiar_registrado  \
27                          60       budismo                  casado   
61                          40  cristianismo             sin_familia   
69                          40  cristianismo               con_hijos   
77                           2  cristianismo                  casado   
106                         32  cristianismo               con_hijos   

     tiene_alto_valor_adquisitivo      trabajo  
27                              1  desempleado  
61                              0  desempleado  
69                              0  desempleado  
77                              0  desempleado  
106                             0  desempleado

observamos que son unas pocas personas las que no completaron ninguno de los dos campos. los volamos?

Ahora verificamos valores negativos en aquellos feautures donde deberian ser todos positivos.

In [180]:
#ganancia perdida que onda?
columnas_con_numeros = ['edad', 'horas_trabajo_registradas']
print('con numeros negativos:')
(df[columnas_con_numeros] < 0).any()
#agregar chequeo de que en estas columnas solo hayan numeros

con numeros negativos:


edad                         False
horas_trabajo_registradas    False
dtype: bool

Vemos que no hay valores negativos en las edades ni en las horas trabajadas. (Nos falta chequear los valores que son numeros que no tengan strings)

Ahora verificamos si hay valores duplicados.

In [181]:
df[df.duplicated(keep=False)]

anios_estudiados   barrio     categoria_de_trabajo  edad  \
12                   17  Palermo  relacion_de_dependencia    23   
17                   13  Palermo  relacion_de_dependencia    32   
20                   20  Palermo  relacion_de_dependencia    40   
26                   13  Palermo  relacion_de_dependencia    19   
29                   13  Palermo  relacion_de_dependencia    49   
...                 ...      ...                      ...   ...   
32503                13  Palermo  relacion_de_dependencia    47   
32512                13  Palermo  relacion_de_dependencia    20   
32527                14  Palermo  relacion_de_dependencia    22   
32557                13  Palermo  relacion_de_dependencia    40   
32559                13  Palermo  relacion_de_dependencia    22   

      educacion_alcanzada    estado_marital  \
12     universidad_3_anio    sin_matrimonio   
17                 4_anio    sin_matrimonio   
20     universiada_6_anio  matrimonio_civil   
26                 4_anio    sin_matrimonio   
29                 4_anio  matrimonio_civil   
...                   ...               ...   
32503              4_anio  matrimonio_civil   
32512              4_anio    sin_matrimonio   
32527              5_anio    sin_matrimonio   
32557              4_anio  matrimonio_civil   
32559              4_anio    sin_matrimonio   

       ganancia_perdida_declarada_bolsa_argentina  genero  \
12                                              0   mujer   
17                                              0  hombre   
20                                              0  hombre   
26                                              0  hombre   
29                                              0  hombre   
...                                           ...     ...   
32503                                           0  hombre   
32512                                           0  hombre   
32527                                           0  hombre   
32557                                           0  hombre   
32559                                           0  hombre   

       horas_trabajo_registradas      religion rol_familiar_registrado  \
12                            30  cristianismo               con_hijos   
17                            40  cristianismo               soltero_a   
20                            60  cristianismo                  casado   
26                            40  cristianismo               con_hijos   
29                            40  cristianismo                  casado   
...                          ...           ...                     ...   
32503                         40  cristianismo                  casado   
32512                         40  cristianismo               con_hijos   
32527                         40  cristianismo               con_hijos   
32557                         40  cristianismo                  casado   
32559                         20  cristianismo               con_hijos   

       tiene_alto_valor_adquisitivo                    trabajo  
12                                0            entretenimiento  
17                                0                  inspector  
20                                1  profesional_especializado  
26                                0                  reparador  
29                                0                  reparador  
...                             ...                        ...  
32503                             0                  reparador  
32512                             0                  inspector  
32527                             0            entretenimiento  
32557                             1                  inspector  
32559                             0            entretenimiento  

[5525 rows x 13 columns]

Podemos ver que hay varias columnas repetidas. Vemos que los valores que se repiten con mas frecuencia son el barrio donde reside cada persona, su categoria de trabajo, la religion y la perdida de ganancia. Concluimos que, dado que el resto de los datos varia, se trata de distintas personas.


## Conversion de variables  
Convertimos aquellas variables categoricas, ya sean ordinales y no ordinales. Empezamos por aquellas variables ordinales, las cuales son:  
    - First item educacion alcanzada

In [59]:
unique_educacion_alcanzada = df['educacion_alcanzada'].value_counts(dropna=False)
oe = OrdinalEncoder()
columns_to_encode = ['educacion_alcanzada']
try:
    df[['educacion_alcanzada_encoded']] = oe.fit_transform(
        df[columns_to_encode]
    )
except Exception as upa:
    print(upa)

print(unique_educacion_alcanzada)
df['educacion_alcanzada_encoded']

4_anio                10501
5_anio                 7291
universidad_3_anio     5355
universidad_4_anio     1723
universidad_1_anio     1382
2_anio                 1175
universidad_2_anio     1067
1_anio                  933
7-8_grado               646
universiada_5_anio      576
9_grado                 514
3_anio                  433
universiada_6_anio      413
5-6_grado               333
1-4_grado               168
preescolar               51
Name: educacion_alcanzada, dtype: int64


0        14.0
1        14.0
2         4.0
3         2.0
4        14.0
         ... 
32556    13.0
32557     4.0
32558     4.0
32559     4.0
32560     4.0
Name: educacion_alcanzada_encoded, Length: 32561, dtype: float64

Ahora vemos que las variables categoricas no ordinales son:
    -barrio
    -categoria_de_trabajo
    -

In [33]:
# Usando pandas profiling
report = ProfileReport(df, title='impuestos', explorative=True, lazy=False)
report

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

/home/sol/.local/lib/python3.8/site-packages/pandas_profiling/visualisation/plot.py:154: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("RdBu"))
  cmap.set_bad(cmap_bad)


Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
#Buscamos entender cada uno de los datos, cómo se relacionan entre sí y si se puede sacar alguna conclusión
#A partir de ellos vamos a determinar los factores que determinan si alguien tiene altos o bajos ingresos

# Visualizacion de datos   

## Primero nos hacemos una serie de preguntas para comprender y analizar mejor nuestros datos:  

influye la edad y el genero de 



#Primero nos preguntamos cuales son los datos que nos van a ayudar a determinar lo pedido
#Para eso nos hacemos una serie de preguntas:

#DUDA: la educacion alcanzada-> cuando no aclara que tipo de educacion que hacemos.
#DUDA: hacer el promedio de cuantos anios dura una carrera, para tomar en cuenta sien funcion de su tiempo de estudio termino o no

#Influye la edad y el genero de la persona?

#Y sus  anios de educacion junto con el nivel de educacion alcanzado?

#Su estado marital junto con su religion? (posiblemente no)

#Vemos que los datos que pueden estar relacionados en cuanto a gastos mensuales son: barrio, rol familiar, o de la perdida en la Esto nos puede ayudar a 
#concluir si una persona tiene un alto poder adquisitivo.

#cuantas horas trabaja en relacion al trabajo que realiza y su categoria. (depende de la categoria lo que te sacan de impuestos)

#vemos que tenemos datos como la religion que no aporta informacion acerca del poder asquisitivo de una persona (consultar
#si se pueden volar los datos sin graficarlos)